# Core

> Core module, with the main `Class` `ConsultaFipe` implementation.

In [ ]:
#| default_exp core

In [ ]:
#| hide
from nbdev.showdoc import *

Path('/Users/joaonogueira/Desktop/Github/fipy/nbs/..')

In [ ]:
#| export
import os
import json
import requests
import pandas as pd

class ConsultaFipe:
    def __init__(self, mes: str = 'agosto/2022', codigo_tipo_veiculo: str = '1'):
        self.mes = mes
        self.codigo_tipo_veiculo = codigo_tipo_veiculo
        self._base_url = "http://veiculos.fipe.org.br/api/veiculos/"
        os.makedirs('datasets', exist_ok=True)

    def save_tables(self):
        mes2 = self.mes.replace('/', '_')
        if not os.path.exists(f'datasets/tabela_referencia_{mes2}.csv'):
            print("Consultando tabela de referência...")
            self.tabela_referencia = self.consulta_tabela_referencia()
            self.tabela_referencia.to_csv(f'datasets/tabela_referencia_{mes2}.csv', index=False)
            print("Tabela de referência salva!")
        else:
            self.tabela_referencia = pd.read_csv(f'datasets/tabela_referencia_{mes2}.csv')
            print(f"Tabela de referência já existe: tabela_referencia_{mes2}.csv")

        if not os.path.exists(f'datasets/tabela_marcas_{mes2}.csv'):
            print("Consultando tabela de marcas...")
            self.tabela_marcas = self.consulta_tabela_marcas()
            self.tabela_marcas.to_csv(f'datasets/tabela_marcas_{mes2}.csv', index=False)
            print("Tabela de marcas salva com sucesso!")
        else:
            self.tabela_marcas = pd.read_csv(f'datasets/tabela_marcas_{mes2}.csv')
            print(f"Tabela de marcas já existe: tabela_marcas_{mes2}.csv")

        if not os.path.exists(f'datasets/tabela_modelos_{mes2}.csv'):
            print('Consultando tabela de modelos...')
            self.tabela_modelos = self.consulta_tabela_modelos()
            self.tabela_modelos.to_csv(f'datasets/tabela_modelos_{mes2}.csv', index=False)
            print('Tabela de modelos salva com sucesso!')
        else:
            self.tabela_modelos = pd.read_csv(f'datasets/tabela_modelos_{mes2}.csv')
            print(f"Tabela de modelos já existe: tabela_modelos_{mes2}.csv")

        if not os.path.exists(f'datasets/tabela_ano_modelo_{mes2}.csv'):
            print("Consultando tabela de ano modelo...")
            self.tabela_ano_modelo = self.consulta_tabela_ano_modelo()
            self.tabela_ano_modelo.to_csv(f'datasets/tabela_ano_modelo_{mes2}.csv', index=False)
            print('Tabela de ano modelo salva com sucesso!')
        else:
            self.tabela_ano_modelo = pd.read_csv(f'datasets/tabela_ano_modelo_{mes2}.csv')
            print(f"Tabela de ano modelo já existe: tabela_ano_modelo_{mes2}.csv")

    def consulta_tabela_referencia(self):
        url = f"{self._base_url}/ConsultarTabelaDeReferencia"
        req = requests.post(url)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json)
        df.columns = ['codigo_mes', 'mes']
        df['mes'] = df['mes'].str.strip()

        return df

    def consulta_tabela_marcas(self):
        url = f"{self._base_url}/ConsultarMarcas"
        codigo_tabela_referencia = self.tabela_referencia.query(f"mes == '{self.mes}'").loc[:,'codigo_mes'][0]
        body = {"codigoTabelaReferencia": codigo_tabela_referencia, "codigoTipoVeiculo": self.codigo_tipo_veiculo}
        req = requests.post(url, data=body)
        req_text = req.text
        req_json = json.loads(req_text)
        df = pd.DataFrame(req_json).rename(columns={'Label': 'marca', 'Value': 'codigo_marca'})
        df['mes'] = self.mes
        return df

    def consulta_tabela_modelos(self):
        url = f"{self._base_url}/ConsultarModelos"
        codigo_tabela_referencia = self.tabela_referencia.query(f"mes == '{self.mes}'").loc[:,'codigo_mes'][0]
        df = pd.DataFrame()
        for codigo_marca in self.tabela_marcas['codigo_marca']:
            body = {"codigoTabelaReferencia": codigo_tabela_referencia, "codigoTipoVeiculo": self.codigo_tipo_veiculo, "codigoMarca": codigo_marca}
            req = requests.post(url, data=body)
            req_text = req.text
            req_json = json.loads(req_text)
            df_temp = pd.DataFrame(req_json['Modelos']).rename(columns={'Label': 'modelo', 'Value': 'codigo_modelo'})
            df_temp['codigo_marca'] = codigo_marca
            df = pd.concat([df, df_temp])
        df['mes'] = self.mes
        return df

    def consulta_tabela_ano_modelo(self):
        url = f"{self._base_url}/ConsultarAnoModelo"
        codigo_tabela_referencia = self.tabela_referencia.query(f"mes == '{self.mes}'").loc[:,'codigo_mes'][0]
        df = pd.DataFrame()
        for codigo_marca in self.tabela_marcas['codigo_marca']:
            for codigo_modelo in self.tabela_modelos.query(f"codigo_marca == '{codigo_marca}'").loc[:,'codigo_modelo']:
                body = {"codigoTabelaReferencia": codigo_tabela_referencia, "codigoTipoVeiculo": self.codigo_tipo_veiculo, "codigoMarca": codigo_marca, "codigoModelo": codigo_modelo}
                req = requests.post(url, data=body)
                print(req.status_code)
                req_text = req.text
                req_json = json.loads(req_text)
                df_temp = pd.DataFrame(req_json).rename(columns={'Label': 'combustivel', 'Value': 'ano'})
                df_temp['codigo_marca'] = codigo_marca
                df_temp['codigo_modelo'] = codigo_modelo
                df = pd.concat([df, df_temp])
        df['mes'] = self.mes
        return df

In [ ]:
url = 'https://veiculos.fipe.org.br/api/veiculos/ConsultarAnoModelo'
body = {"codigoTabelaReferencia": 288, "codigoTipoVeiculo": 1, "codigoMarca": 59, "codigoModelo": 9564}
req = requests.post(url, data=body)

In [ ]:
req.text

'[{"Label":"32000 Gasolina","Value":"32000-1"},{"Label":"2022 Gasolina","Value":"2022-1"},{"Label":"2021 Gasolina","Value":"2021-1"},{"Label":"2020 Gasolina","Value":"2020-1"}]'

In [ ]:
fipe = ConsultaFipe(mes='agosto/2022')

In [ ]:
fipe.save_tables()

Tabela de referência já existe: tabela_referencia_agosto_2022.csv
Tabela de marcas já existe: tabela_marcas_agosto_2022.csv
Tabela de modelos já existe: tabela_modelos_agosto_2022.csv
Tabela de ano modelo já existe: tabela_ano_modelo_agosto_2022.csv


In [ ]:
fipe.consulta_tabela_ano_modelo()

,mes


In [ ]:
fipe.tabela_referencia.head()

,codigo_mes,mes
0,288,agosto/2022
1,287,julho/2022
2,286,junho/2022
3,285,maio/2022
4,284,abril/2022


In [ ]:
fipe.tabela_marcas.head()

,marca,codigo_marca,mes
0,Acura,1,agosto/2022
1,Agrale,2,agosto/2022
2,Alfa Romeo,3,agosto/2022
3,AM Gen,4,agosto/2022
4,Asia Motors,5,agosto/2022


In [ ]:
fipe.tabela_marcas[fipe.tabela_marcas['marca'].str.contains('VW')]

,marca,codigo_marca,mes
89,VW - VolksWagen,59,agosto/2022


In [ ]:
fipe.tabela_modelos.head()

,modelo,codigo_modelo,codigo_marca,mes
0,Integra GS 1.8,1,1,agosto/2022
1,Legend 3.2/3.5,2,1,agosto/2022
2,NSX 3.0,3,1,agosto/2022
0,MARRUÁ 2.8 12V 132cv TDI Diesel,4,2,agosto/2022
1,MARRUÁ AM 100 2.8 CS TDI Diesel,4564,2,agosto/2022


In [ ]:
#| hide
import nbdev
nbdev.nbdev_export()